#MOVIE RECCOMENDER SYSTEM

####-Content based
####-Collaborative filtering
####-Hybrid

In [831]:
import pandas as pd
import numpy as np

movies=pd.read_csv("tmdb_5000_movies.csv")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [832]:
credits=pd.read_csv("tmdb_5000_credits.csv")
credits = credits.iloc[:,:4]
credits.head(1)


C:\Users\Tarun F Rekhraj\AppData\Local\Temp\ipykernel_20420\3846445618.py:1: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


#####merging data frames with help of movie_id /title [PRIMARY KEY]

In [833]:
df=movies.merge(credits, on="title") #there should be 23 columns only as 2 diff title columns become one single column
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


#IMP THINGS FOR RECOMMENDATION:-
###-genres , id , keyword , title , overview , cast , crew



In [834]:
df=df[['id','title','genres','overview','keywords','cast','crew']]
df.head(1)

,id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [835]:
df.dropna(inplace=True) #remove null values
df.isnull().sum()

id          0
title       0
genres      0
overview    0
keywords    0
cast        0
crew        0
dtype: int64

In [836]:
#check duplicates
df.duplicated().sum()  #no duplicates  if it were there use drop_duplicates()

0

## to make genres, keyword, cast, crew look ideal 

In [837]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [838]:
df['genres']=df['genres'].apply(convert)
df['keywords']=df['keywords'].apply(convert)

In [839]:
# The 'cast' column already contains lists of strings, so no need to apply 'convert' again.
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L
# just used 3 casts from each movie as they are only the mains through which recommendation will help

df['cast'] = df['cast'].apply(convert3)

In [840]:
# Extracting the director from the crew column
def fetch_director(obj):
    L = []
    try:
        for i in ast.literal_eval(obj):
            if i['job'] == "Director":  
                L.append(i['name'])
                break
    except (ValueError, SyntaxError):
        # Handle cases where the string cannot be parsed
        pass
    return L

df['crew'] = df['crew'].apply(fetch_director)

In [841]:
df.head(3)

,id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]


###To remove spaces between sentences

In [842]:
df['genres']=df['genres'].apply(lambda x:[i.replace(" ","")for i in x])
df['keywords']=df['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
df['cast']=df['cast'].apply(lambda x:[i.replace(" ","")for i in x])
df['crew']=df['crew'].apply(lambda x:[i.replace(" ","")for i in x])
df.head(3)

,id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","In the 22nd century, a paraplegic Marine is di...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]


####Catenation to become 1 tag

In [843]:
#df['tags']=df['genres'] + df['overview'] + df['keywords'] + df['cast'] + df['crew']
#we could have done this but they are not strings they are List
df['tags'] = df['genres'] + df['keywords'] + df['cast'] + df['crew'] + df['overview'].apply(lambda x: x.split())
df.head(3)


,id,title,genres,overview,keywords,cast,crew,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","In the 22nd century, a paraplegic Marine is di...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime, spy, basedonnovel, ..."


In [844]:
new_df= df[['id','title','tags']]
new_df.head(4)

,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."


In [845]:
df['tags'].values
#to join all tags
new_df['tags'].apply(lambda x:" ".join(x))  # all tags everythiing became 1

0       Action Adventure Fantasy ScienceFiction cultur...
1       Adventure Fantasy Action ocean drugabuse exoti...
2       Action Adventure Crime spy basedonnovel secret...
3       Action Crime Drama Thriller dccomics crimefigh...
4       Action Adventure ScienceFiction basedonnovel m...
                              ...                        
4804    Action Crime Thriller unitedstates–mexicobarri...
4805    Comedy Romance EdwardBurns KerryBishé MarshaDi...
4806    Comedy Drama Romance TVMovie date loveatfirsts...
4807    DanielHenney ElizaCoupe BillPaxton DanielHsia ...
4808    Documentary obsession camcorder crush dreamgir...
Name: tags, Length: 4802, dtype: object

In [846]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x).lower())

C:\Users\Tarun F Rekhraj\AppData\Local\Temp\ipykernel_20420\274634086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x).lower())


In [847]:
new_df.head()

,id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


# VECTORIZATION

In [848]:
# convert TEXT to VECTOR/ NUMERIC FORM by knowing how many times a word is repeated & then find out closest vectors to recommend movies

In [849]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000, stop_words='english')  # stop_words means to remove all english words
cv.fit_transform(new_df['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [850]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [851]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

####using STEM to change loving,loved... to love

In [852]:
import nltk
from nltk.stem.porter import PorterStemmer  
ps=PorterStemmer()

def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i)) # did stem of all words and saved in y
    return " ".join(y)

new_df['tags'].apply(stem)

0       action adventur fantasi sciencefict culturecla...
1       adventur fantasi action ocean drugabus exotici...
2       action adventur crime spi basedonnovel secreta...
3       action crime drama thriller dccomic crimefight...
4       action adventur sciencefict basedonnovel mar m...
                              ...                        
4804    action crime thriller unitedstates–mexicobarri...
4805    comedi romanc edwardburn kerrybishé marshadiet...
4806    comedi drama romanc tvmovi date loveatfirstsig...
4807    danielhenney elizacoup billpaxton danielhsia w...
4808    documentari obsess camcord crush dreamgirl dre...
Name: tags, Length: 4802, dtype: object

###To calculate distance of each points using angle theta

In [853]:
from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(vectors)  # passing all vectors in cosine graph so that we know which points are near to each other [in 4802*4802 table]
#diagnols are 1 as it shows similarity of 1st movie with 1st movie, 2nd with 2nd.......
similarity

array([[1.        , 0.08980265, 0.05892557, ..., 0.02512595, 0.02777778,
        0.        ],
       [0.08980265, 1.        , 0.06350006, ..., 0.02707652, 0.        ,
        0.        ],
       [0.05892557, 0.06350006, 1.        , ..., 0.02665009, 0.        ,
        0.        ],
       ...,
       [0.02512595, 0.02707652, 0.02665009, ..., 1.        , 0.07537784,
        0.0489116 ],
       [0.02777778, 0.        , 0.        , ..., 0.07537784, 1.        ,
        0.05407381],
       [0.        , 0.        , 0.        , ..., 0.0489116 , 0.05407381,
        1.        ]])

#####sort and recommend movies

In [854]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]  #enumerate=to hold similarities of 1st movie with 1st,2nd...
#list=creates a proper tuple for showcasing enumerate    ,    reverse= start from most similar recommendation    ,    key lambda=for reversing wrt right side ie. 0.257     ,     [1:5]= donates no. of items similar u want to showcase excludes 0th movie as 0-0 similarity will be 1

[(538, 0.25724787771376323),
 (1189, 0.2545875386086578),
 (506, 0.24948506639458295),
 (1211, 0.24459979523511427),
 (581, 0.2405626121623441)]

In [855]:
def recommend(movie):
    movie_index= new_df[new_df['title']==movie]. index[0]  #index 0 means for first movie AVATAR
    distances= similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]  # over here its distances and  not similarity
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [856]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [857]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Aliens vs Predator: Requiem
Battle: Los Angeles


In [858]:
# to data and paste in app.py for recommendation search
import pickle
pickle.dump(new_df.to_dict(), open('movies_dict.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))